In [1]:
import os
import json
import pandas as pd

각 알약 조합 별, 모든 알약에 대해 정보가 존재하는 이미지들을 구한다.

In [2]:
images=sorted(os.listdir('./ai06-level1-project/train_images'))
annot_root='./ai06-level1-project/train_annotations'
annots_t=sorted(os.listdir(annot_root)) # 알약 조합 목록
annots_list=[]
images_list=[]
# 각 알약 조합에 대해
for annot_set in annots_t:
    # 약 목록을 구한다.
    pill_list=sorted(os.listdir(os.path.join(annot_root,annot_set)))
    # 첫 약이 가지는 json 파일 목록 (각 이미지에 대한 json 파일 목록)
    base_images=sorted(os.listdir(os.path.join(annot_root,annot_set,pill_list[0])))
    # 각 약에 대해
    for pill in pill_list:
        # 각 약이 어떤 이미지에 대한 annotation을 갖는지에 대한 list
        imgs=sorted(os.listdir(os.path.join(annot_root,annot_set,pill)))
        # 특정 약은 annotation이 없는 이미지여도 추가
        for img in imgs:
            if img not in base_images:
                base_images.append(img)
    # 모든 약에 대해, 공통으로 json 파일이 있는 이미지의 이름만 저장
    for image in base_images:
        annots_list.append(image[:-5])
# 
for img in images:
    images_list.append(img[:-4])
len(annots_list), annots_list[0], len(images_list),images_list[0]

(369,
 'K-001900-016548-019607-029451_0_2_0_2_70_000_200',
 651,
 'K-001900-010224-016551-031705_0_2_0_2_70_000_200')

전체 파일 중, annotation이 하나라도 있는 이미지들의 이름만 추출

In [3]:
images_set=set(images_list)
annots_set=set(annots_list)
# 이미지에 대해, annotation이 있는 경우의 이름들을 모음
available_files=sorted(list(images_set.intersection(annots_set)))
len(available_files)

232

In [4]:
with open('./annots_without_image.txt','w') as f:
    for annot in annots_list:
        if annot not in images_list:
            f.write(str(annot)+'\n')

In [5]:
with open('./images_without_annot.txt','w') as f:
    for image in images_list:
        if image not in annots_list:
            f.write(str(image)+'\n')

In [6]:
for f in available_files:
    print(f)
    break

K-001900-016548-019607-029451_0_2_0_2_70_000_200


이미지 단위로 annotation 파일들을 결합합니다.</br>
이때, 추후 추가 데이터를 사용하기 위해서 image의 id 목록과 annot의 id 목록을 따로 얻습니다.

In [7]:
annot_ids=[]
image_ids=[]

In [8]:
#json 파일 통합

base_path = "./ai06-level1-project"
ann_root = os.path.join(base_path, "train_annotations")

output_path = os.path.join(base_path, "train_annots")
os.makedirs(output_path, exist_ok=True)

image_id_offset = 0
annotation_id_offset = 0
category_set = set()

# 알약 조합 별 폴더 명
for root in sorted(os.listdir(ann_root)):

    # 현재 조합의 알약 종류
    pills=sorted(os.listdir(os.path.join(ann_root,root)))

    

    # 현재 조합의 이미지 목록
    images_list=[]
    for pill in pills:
        images_list_p=os.listdir(os.path.join(ann_root,root,pill))
        for i in images_list_p:
            if i not in images_list:
                images_list.append(i)

    if len(images_list)==0:
        continue

    # 각 이미지 별 JSON 파일 생성
    for img in images_list:

        images=[]
        annotations=[]
        categories=[]

        # 대응하는 이미지가 없거나, 일부 알약에 대해 누락되어 있다 annotation이 누락된 이미지면 continue
        if img[:-5] not in available_files:
            continue
        # 각 알약에 대해서
        for pill in pills:
            # 만약 이번 알약은 해당 이미지에 대한 annotation이 없다면 넘어갑니다.
            if img not in os.listdir(os.path.join(ann_root,root,pill)):
                continue
            # 해당 이미지의 json 파일을 엽니다.
            with open(str(os.path.join(ann_root,root,pill,img)), "r", encoding="utf-8") as f:
                pill_data = json.load(f)
                if len(images)==0:
                    images.extend(pill_data["images"])
                    image_ids.append(pill_data['images'][0]['id'])
                annotations.extend(pill_data["annotations"])
                annot_ids.append(pill_data['annotations'][0]['id'])
                categories.extend(pill_data["categories"])
        output_coco={
            "images": images,
            "annotations": annotations,
            "categories": categories
        }
        with open(os.path.join(output_path,img), "w", encoding="utf-8") as f:
            json.dump(output_coco, f, ensure_ascii=False, indent=4)

In [9]:
len(image_ids), len(annot_ids)

(232, 763)

In [10]:
type(image_ids[0]),type(annot_ids[0])

(int, int)

추가 데이터는 max 값을 기준으로 id를 주겠습니다.

In [11]:
max(image_ids),max(annot_ids)

(1499, 5691)

혹시 모르니 중복 값을 확인합니다.

In [12]:
len(set(image_ids)),len(set(annot_ids))

(232, 763)

이제 모든 이미지의 json 파일을 결합합니다.

In [13]:
output_all_coco = {
    "images": [],
    "annotations": [],
    "categories": []
}
files = os.listdir(os.path.join(base_path, "train_annots"))

for file in files:
    with open(os.path.join(base_path, "train_annots", file), "r", encoding="utf-8") as f:
        data=json.load(f)
        for image in data['images']:
            output_all_coco['images'].append(image)
        for annot in data['annotations']:
            output_all_coco['annotations'].append(annot)
        for cat in data['categories']:
            output_all_coco['categories'].append(cat)

t=pd.DataFrame(output_all_coco['categories'])
t=t.drop_duplicates()
output_all_coco['categories']=t.to_dict(orient='records')

save_path = os.path.join(base_path, "train.json")
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(output_all_coco, f, ensure_ascii=False, indent=4)

print("train.json 생성됨:", save_path)

train.json 생성됨: ./ai06-level1-project/train.json


In [14]:
len(output_all_coco['categories'])

56

In [15]:
len(output_all_coco['images'])

232

In [16]:
len(output_all_coco['annotations'])

763

In [17]:
output_all_coco['categories']

[{'supercategory': 'pill', 'id': 1899, 'name': '보령부스파정 5mg'},
 {'supercategory': 'pill', 'id': 16547, 'name': '가바토파정 100mg'},
 {'supercategory': 'pill', 'id': 19606, 'name': '스토가정 10mg'},
 {'supercategory': 'pill', 'id': 29450, 'name': '레일라정'},
 {'supercategory': 'pill', 'id': 33008, 'name': '신바로정'},
 {'supercategory': 'pill', 'id': 21770, 'name': '라비에트정 20mg'},
 {'supercategory': 'pill', 'id': 27925, 'name': '울트라셋이알서방정'},
 {'supercategory': 'pill', 'id': 24849, 'name': '놀텍정 10mg'},
 {'supercategory': 'pill', 'id': 29344, 'name': '비모보정 500/20mg'},
 {'supercategory': 'pill', 'id': 16550, 'name': '동아가바펜틴정 800mg'},
 {'supercategory': 'pill', 'id': 33207, 'name': '에스원엠프정 20mg'},
 {'supercategory': 'pill', 'id': 2482, 'name': '뮤테란캡슐 100mg'},
 {'supercategory': 'pill', 'id': 3742, 'name': '알드린정'},
 {'supercategory': 'pill', 'id': 12777, 'name': '다보타민큐정 10mg/병'},
 {'supercategory': 'pill', 'id': 13394, 'name': '써스펜8시간이알서방정 650mg'},
 {'supercategory': 'pill', 'id': 12080, 'name': '리렉스펜정 300mg/